# Advent of Code - 2024 - Day 20 - Problem 1

https://adventofcode.com/2024/day/20

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day20.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

DATA = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""
DATA = list(map(str.strip, DATA.splitlines()))

DATA

['###############',
 '#...#...#.....#',
 '#.#.#.#.#.###.#',
 '#S#...#.#.#...#',
 '#######.#.#.###',
 '#######.#.#...#',
 '#######.#.###.#',
 '###..E#...#...#',
 '###.#######.###',
 '#...###...#...#',
 '#.#####.#.###.#',
 '#.#...#.#.#...#',
 '#.#.#.#.#.#.###',
 '#...#...#...###',
 '###############']

## Create Map Class

In [2]:
class Map:

    def __init__(self, lines):
        self._lines = list(map(list, DATA))
        self._rows = len(self._lines)
        self._cols = len(self._lines[0])

        # Find the starting position
        #
        for row in range(self._rows):
            for col in range(self._cols):
                if self._lines[row][col] == "S":
                    self._start = (row, col)
                if self._lines[row][col] == "E":
                    self._end = (row, col)

    def can_move(self, target, cheat):
        if cheat:
            return target == "#"
        else:
            return target != "#"

    def get_next_positions(self, position, cheat):
        row, col = position
        if row > 0:
            target = self._lines[row - 1][col]
            if self.can_move(target, cheat):
                yield (row - 1, col)
        if row < self._rows - 1:
            target = self._lines[row + 1][col]
            if self.can_move(target, cheat):
                yield (row + 1, col)
        if col > 0:
            target = self._lines[row][col - 1]
            if self.can_move(target, cheat):
                yield (row, col - 1)
        if col < self._cols - 1:
            target = self._lines[row][col + 1]
            if self.can_move(target, cheat):
                yield (row, col + 1)

    def find_paths(self, path):

        last_position = path[-1]
        for next_position in self.get_next_positions(last_position, False):
            if not next_position in path:
                new_path = list(path)
                new_path.append(next_position)
                if next_position == self._end:
                    yield new_path
                else:
                    yield from self.find_paths(new_path)                

    def find_cheat_paths(self, path, max_length):

        last_position = path[-1]
        for next_position in self.get_next_positions(last_position, True):
            if not next_position in path:
                new_path = list(path)
                new_path.append(next_position)
                yield new_path
                if len(new_path) < max_length:
                    yield from self.find_cheat_paths(new_path, max_length)                

    def find_pathsx(self, path, cheat_length, path_cache):

        # print(f"find_paths - {path}")
        must_cheat = False
        can_cheat = False
        if cheat_length == len(path):
            must_cheat = True
            # print("must_cheat")
        elif cheat_length == len(path) -1:
            pass
            # can_cheat = True
            # print("can_cheat")
        cheated = len(path) > cheat_length

        last_position = path[-1]
        if cheated and last_position in path_cache:
            new_path = list(path)
            new_path.extend(path_cache[last_position])
            yield new_path
        else:
            for next_position in self.get_next_positions(last_position, must_cheat, can_cheat):
                if not next_position in path:
                    # if next_position in distances_to_exit and distances_to_exit[next_position] + len(path) > 9372:
                    #     pass
                    # else:
                    new_path = list(path)
                    new_path.append(next_position)
                    # print(next_position)
                    if next_position == self._end:
                        # print("end found")
                        yield new_path
                    else:
                        yield from self.find_paths(new_path, cheat_length, path_cache)

        # last_position = path[-1]
        # cheating = cheat_length <= len(path) <= cheat_length + 1
        # # was_cheating = cheat_length + 1 < len(path)

        # # do_pop = False
        # for next_position in self.get_next_positions(last_position, cheating):
        #     # if was_cheating:
        #     #     if next_position not in distances_to_exit: return
        #     #     if len(path) + 1 + distances_to_exit[next_position] > 10000: return

        #     if not next_position in path:
        #         new_path = list(path)
        #         new_path.append(next_position)
        #         # if do_pop: path.pop(-1)
        #         # do_pop = True
        #         # path.append(next_position)
        #         if next_position == self._end: yield new_path
        #         else: yield from self.find_paths(new_path, cheat_length, distances_to_exit)

    def print(self, path):
        m = list()
        for line in self._lines:
            m.append(list(line))

        if path != None:
            for location in path:
                row, col, is_cheat = location
                m[row][col] = "*"

        for line in m:
            print("".join(line))

## Create Path Class

In [3]:
class Path:

    def __init__(self, position):
        self._position = position

## Find Solutions

In [4]:
import sys

sys.setrecursionlimit(10000)

m = Map(DATA)
# m.print(None)
# print(m._end)

starting_path = [m._start]

# Find the path without cheating
#
for path in m.find_paths(starting_path):
    honest_path = path
    break
print(f"honest_path = {honest_path}")

honest_path_length = len(honest_path)
print(f"honest_path_length = {honest_path_length}")

lengths_to_exit = dict()
for idx in range(len(honest_path)):
    location = honest_path[idx]
    length_to_exit = honest_path_length - idx
    lengths_to_exit[location] = length_to_exit
print(f"lengths_to_exit = {lengths_to_exit}")

improvements = dict()
for idx in range(len(honest_path)):
    position = honest_path[idx]
    exits = dict()
    for cheat_path in m.find_cheat_paths([position], 19):
        position_cheat = cheat_path[-1]
        for position_honest in m.get_next_positions(position_cheat, False):
            length_to_exit= lengths_to_exit[position_honest]
            cheat_length = idx + len(cheat_path) + length_to_exit
            improvement = honest_path_length - cheat_length
            if improvement >= 50:
                if position_honest in exits:
                    current_improvement = exits[position_honest]
                    if improvement > current_improvement:
                        exits[position_honest] = improvement
                        if position_honest == (7,3):
                            print(f"cheat_path = {cheat_path}")
                            print(f"position_cheat = {position_cheat}")
                            print(f"position_honest = {position_honest}")
                            print(f"cheat_length = {cheat_length}")
                            print(f"improvement = {improvement}")
                else:
                    exits[position_honest] = improvement
                    if position_honest == (7,3):
                        print(f"cheat_path = {cheat_path}")
                        print(f"position_cheat = {position_cheat}")
                        print(f"position_honest = {position_honest}")
                        print(f"cheat_length = {cheat_length}")
                        print(f"improvement = {improvement}")

    print(f"idx = {idx}, position = {position} exits = {exits}")    
    for position in exits:
        improvement = exits[position]
        if improvement in improvements:
            improvements[improvement] += 1
        else:
            improvements[improvement] = 1
print(f"improvements = {improvements}")

total_count = 0
for improvement in improvements:
    total_count += improvements[improvement]
print(f"total_count = {total_count}")

# print(f"honest_path = {honest_path}")

# honest_length = len(honest_path) - 1
# print(f"honest_length = {honest_length}")

# distances_to_exit = dict()
# distance_to_exit = honest_length
# for position in honest_path:
#     distances_to_exit[position] = distance_to_exit
#     distance_to_exit -= 1

# print(f"distances_to_exit = {distances_to_exit}")

# count = 0
# improvements = dict()
# for cheat_length in range(1, honest_length):  # range(1, honest_length):
#     for path in m.find_paths(starting_path, cheat_length, path_cache):
#         path_length = len(path) - 1
#         # if len(path) < honest_length - 1:
#         improvement = honest_length - path_length
#         if improvement >= 100: #== 64:
#             if improvement in improvements:
#                 improvements[improvement] += 1
#             else:
#                 improvements[improvement] = 1
#             count += 1
#             print("--------------------------------------------")
#             print(
#                 f"cheat_length = {cheat_length}, path_length={path_length}, improvement = {improvement}, count = {count}"
#             )
#             break
#             # print(path)
#             # m.print(path)

# print(count)
# print(improvements)

honest_path = [(3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (7, 8), (7, 9), (6, 9), (5, 9), (4, 9), (3, 9), (2, 9), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (2, 13), (3, 13), (3, 12), (3, 11), (4, 11), (5, 11), (5, 12), (5, 13), (6, 13), (7, 13), (7, 12), (7, 11), (8, 11), (9, 11), (9, 12), (9, 13), (10, 13), (11, 13), (11, 12), (11, 11), (12, 11), (13, 11), (13, 10), (13, 9), (12, 9), (11, 9), (10, 9), (9, 9), (9, 8), (9, 7), (10, 7), (11, 7), (12, 7), (13, 7), (13, 6), (13, 5), (12, 5), (11, 5), (11, 4), (11, 3), (12, 3), (13, 3), (13, 2), (13, 1), (12, 1), (11, 1), (10, 1), (9, 1), (9, 2), (9, 3), (8, 3), (7, 3), (7, 4), (7, 5)]
honest_path_length = 85
lengths_to_exit = {(3, 1): 85, (2, 1): 84, (1, 1): 83, (1, 2): 82, (1, 3): 81, (2, 3): 80, (3, 3): 79, (3, 4): 78, (3, 5): 77, (2, 5): 76, (1, 5): 75, (1, 6): 74, (1, 7): 73, (2, 7): 72, (3, 7): 71, (4, 7): 70, (5, 7): 69, (6,